<div style="text-align: center;">
    <h1><strong>CAPSTONE 3 - MODEL PREDICTION</strong></h1>
    <p><em>ANNISA DITASILA</em></p>
    <p><em>DTIDS - 0106</em></p>
</div>


# **Library**

In [19]:
import pandas as pd
import numpy as np
import pickle

###  Settings for Google Cloud 

In [20]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [21]:

## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../Capstone_3/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'dita_dataset_003'
table_id = 'data_test'
region = 'us-central1'
bucket_name = 'dita_gcs_003'
blob_name = 'model/model_gbc.sav'
model_name = 'model_gbc.sav'

### Google Cloud Storage

### Retrieve from Google Cloud Storage

In [22]:
# Retrieve the model from Google Cloud Storage
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename(model_name)

    print("Read model succeeded")
except Exception as e:
    raise TypeError(f"An exception occurred: {e}")


Read model succeeded


## **Model Prediction using Data-Test (Big Query)**

## Load Model

In [23]:
# Load the model
with open('model_gbc.sav', 'rb') as f:
    loaded_model = pickle.load(f)


### Load from BigQuery

In [24]:
#load data from BQ
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df_test = query_job.result().to_dataframe()

df_test

/home/ditasila/miniconda3/envs/dita-capstone-env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges
0,False,9,Yes,No,Fiber optic,No,No,Month-to-month,True,74.25
1,False,1,No,No,DSL,No,No,Month-to-month,True,35.45
2,False,1,No,No,Fiber optic,No,No,Month-to-month,True,91.30
3,False,8,Yes,No,Fiber optic,No,No,Month-to-month,True,75.00
4,False,5,No,No,Fiber optic,No,No,Month-to-month,False,75.60
...,...,...,...,...,...,...,...,...,...,...
94,False,9,No internet service,No internet service,No,No internet service,No internet service,One year,True,20.15
95,True,53,No internet service,No internet service,No,No internet service,No internet service,One year,True,19.85
96,True,69,No internet service,No internet service,No,No internet service,No internet service,Two year,False,19.40
97,False,18,No internet service,No internet service,No,No internet service,No internet service,One year,False,20.35


In [25]:
df_test['dependents'] = df_test['dependents'].astype(str).map({'True': 'Yes', 'False': 'No'})

In [26]:
df_test['paperlessbilling'] = df_test['paperlessbilling'].astype(str).map({'True': 'Yes', 'False': 'No'})

In [27]:
y_pred_x_test = loaded_model.predict(df_test)

In [28]:
df_test_prediction = df_test.copy()
df_test_prediction['churn'] = y_pred_x_test
df_test_prediction

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges,churn
0,No,9,Yes,No,Fiber optic,No,No,Month-to-month,Yes,74.25,1
1,No,1,No,No,DSL,No,No,Month-to-month,Yes,35.45,1
2,No,1,No,No,Fiber optic,No,No,Month-to-month,Yes,91.30,1
3,No,8,Yes,No,Fiber optic,No,No,Month-to-month,Yes,75.00,1
4,No,5,No,No,Fiber optic,No,No,Month-to-month,No,75.60,1
...,...,...,...,...,...,...,...,...,...,...,...
94,No,9,No internet service,No internet service,No,No internet service,No internet service,One year,Yes,20.15,0
95,Yes,53,No internet service,No internet service,No,No internet service,No internet service,One year,Yes,19.85,0
96,Yes,69,No internet service,No internet service,No,No internet service,No internet service,Two year,No,19.40,0
97,No,18,No internet service,No internet service,No,No internet service,No internet service,One year,No,20.35,0


# **Model Prediction for Single Customer**

In [29]:
# Single Customer Prediction

new_cust = pd.DataFrame([
    {
        'dependents': 'No',
        'tenure': 23,
        'onlinesecurity': 'No',
        'onlinebackup': 'No',
        'internetservice': 'DSL',
        'deviceprotection': 'No',
        'techsupport': 'Yes',
        'contract': 'One year',
        'paperlessbilling': 'No',
        'monthlycharges': 111.90
    },
    {
        'dependents': 'Yes',
        'tenure': 9,
        'onlinesecurity': 'No',
        'onlinebackup': 'No',
        'internetservice': 'DSL',
        'deviceprotection': 'No',
        'techsupport': 'No',
        'contract': 'Month-to-month',
        'paperlessbilling': 'Yes',
        'monthlycharges': 20.15
    }
])

new_cust

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges
0,No,23,No,No,DSL,No,Yes,One year,No,111.90
1,Yes,9,No,No,DSL,No,No,Month-to-month,Yes,20.15


In [30]:
y_pred_cloud_new_data = loaded_model.predict(new_cust)
y_pred_cloud_new_data

array([0, 1])

In [31]:
new_test_cloud = new_cust.copy()
new_test_cloud['churn'] = ['Yes' if pred == 1 else 'No' for pred in y_pred_cloud_new_data]
new_test_cloud

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges,churn
0,No,23,No,No,DSL,No,Yes,One year,No,111.90,No
1,Yes,9,No,No,DSL,No,No,Month-to-month,Yes,20.15,Yes


# **Upload the prediction to BigQuery**

In [32]:
from google.cloud import bigquery

table_id = 'prediction result'

# Construct a BigQuery client object.
client = bigquery.Client()

# Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

df_test_prediction.columns = ['_'.join(i.split(' ')) for i in df_test_prediction.columns] # make sure no empty space on columns' name

# Load the DataFrame into the BigQuery table
job = client.load_table_from_dataframe(df_test_prediction, table_full_id) # your df predicted name, and table full id above

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 99 rows into dti-ds.dita_dataset_003.prediction result
